In [1]:
import gc
import os
import time

import pandas as pd 
from datetime import datetime
import numpy as np
from numba import njit, prange
from itertools import combinations

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.activations import gelu, softmax
from tensorflow.keras.models import Sequential

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow_addons.activations import sparsemax
import random
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam, AdamW
from tensorflow.keras.models import Model, clone_model
from tensorflow.keras.layers import Input, Dropout, Dense, ReLU, BatchNormalization, Activation, Concatenate
from copy import deepcopy

import tensorflow.keras.layers as layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

import warnings
from warnings import simplefilter
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore")
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

import joblib
import functools

import gc

EPS = 1e-8
gc.collect();

2023-12-10 07:02:30.350392: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-10 07:02:30.483743: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-10 07:02:30.483802: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-10 07:02:30.484571: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-10 07:02:30.539045: I tensorflow/core/platform/cpu_feature_g

In [2]:
MODE = "train"  # train, inference, both
KAGGLE_DATASET_NAME = "tabtransformer-version-yongmin-0"

In [3]:
if MODE == "train":
    print("You are in train mode")
    model_directory = "./models/" + time.strftime("%Y%m%d_%H:%M:%S", time.localtime(time.time() + 9 * 60 * 60))
    data_directory = "./data"
    train_mode = True
    infer_mode = False
elif MODE == "inference":
    print("You are in inference mode")
    model_directory = f'/kaggle/input/{KAGGLE_DATASET_NAME}'
    data_directory = "/kaggle/input/optiver-trading-at-the-close"
    train_mode = False
    infer_mode = True
elif MODE == "both":
    print("You are in both mode")
    model_directory = f'/kaggle/working/'
    data_directory = "/kaggle/input/optiver-trading-at-the-close"
    train_mode = True
    infer_mode = True
else:
    raise ValueError("Invalid mode")

You are in train mode


In [4]:
config = {
    "data_dir": data_directory,
    "model_dir": model_directory,

    "train_mode": train_mode,  # True : train, False : not train
    "infer_mode": infer_mode,  # True : inference, False : not inference
    "model_name": ["lgb"],  # model name
    "final_mode": False,  # True : using final model, False : not using final model
    "best_iterate_ratio": 1.2,  # best iteration ratio
    'target': 'target',

    'split_method': 'rolling',  # time_series, rolling, blocking, holdout
    'n_splits': 3,  # number of splits
    'correct': True,  # correct boundary
    'gap': 0.05,  # gap between train and test (0.05 = 5% of train size)

    'initial_fold_size_ratio': 0.8,  # initial fold size ratio
    'train_test_ratio': 0.9,  # train, test ratio

    'optuna_random_state': 42,
}
config["model_mode"] = "single" if len(config["model_name"]) == 1 else "stacking"  # 모델 수에 따라서 single / stacking 판단
config["mae_mode"] = True if config["model_mode"] == "single" and not config[
    "final_mode"] else False  # single 모델이면서 final_mode가 아닌경우 폴드가 여러개일때 모델 평가기준이 없어서 mae로 평가
config["inference_n_splits"] = len(config['model_name']) if config["final_mode"] or config["mae_mode"] else config[
    "n_splits"]  # final_mode가 아닌경우 n_splits만큼 inference

In [5]:
if MODE == "train":
    if not os.path.exists(config["model_dir"]):
        os.makedirs(config["model_dir"])
    if not os.path.exists(config["data_dir"]):
        os.makedirs(config["data_dir"])
    !kaggle competitions download optiver-trading-at-the-close -p {config["data_dir"]} --force
    !unzip -o {config["data_dir"]}/optiver-trading-at-the-close.zip -d {config["data_dir"]}
    !rm {config["data_dir"]}/optiver-trading-at-the-close.zip

 99%|███████████████████████████████████████▍| 198M/201M [00:06<00:00, 29.0MB/s]
100%|████████████████████████████████████████| 201M/201M [00:06<00:00, 31.2MB/s]
Archive:  ./data/optiver-trading-at-the-close.zip
  inflating: ./data/example_test_files/revealed_targets.csv  
  inflating: ./data/example_test_files/sample_submission.csv  
  inflating: ./data/example_test_files/test.csv  
  inflating: ./data/optiver2023/__init__.py  
  inflating: ./data/optiver2023/competition.cpython-310-x86_64-linux-gnu.so  
  inflating: ./data/public_timeseries_testing_util.py  
  inflating: ./data/train.csv        


In [6]:
def print_log(message_format):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            # self 확인: 첫 번째 인자가 클래스 인스턴스인지 확인합니다.
            if args and hasattr(args[0], 'infer'):
                self = args[0]

                # self.infer가 False이면 아무 것도 출력하지 않고 함수를 바로 반환합니다.
                if self.infer:
                    return func(*args, **kwargs)

            start_time = time.time()
            result = func(*args, **kwargs)
            end_time = time.time()

            elapsed_time = end_time - start_time

            if result is not None:
                data_shape = getattr(result, 'shape', 'No shape attribute')
                shape_message = f", shape({data_shape})"
            else:
                shape_message = ""

            print(f"\n{'-' * 100}")
            print(message_format.format(func_name=func.__name__, elapsed_time=elapsed_time) + shape_message)
            print(f"{'-' * 100}\n")

            return result

        return wrapper

    return decorator


In [7]:
def reduce_mem_usage(df, verbose=0):
    """
    Iterate through all numeric columns of a dataframe and modify the data type
    to reduce memory usage.
    """

    start_mem = df.memory_usage().sum() / 1024 ** 2

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)

    return df

In [8]:
class DataPreprocessor:

    def __init__(self, data, infer=False):
        self.data = memory_reduction.reduce_mem_usage(data)
        self.infer = infer

    @print_log("Executed {func.__name__} : shape({self.data.shape})")
    def handle_missing_data(self):
        # 결측치 처리 코드
        self.data = self.data.dropna(subset=["target"]) if self.infer == False else self.data
        self.data = self.data.fillna(0, columns = ['far_price', 'near_price'])
        self.data.reset_index(drop=True, inplace=True)
        return self.data

    @print_log("Executed {func.__name__} : shape({self.data.shape})")
    def handle_outliers(self):
        # 이상치 처리 코드
        return self.data

    @print_log("Executed {func.__name__} : shape({self.data.shape})")
    def normalize(self):
        # 정규화 코드
        return self.data

    @print_log("Executed {func.__name__} : shape({self.data.shape})")
    def custom_preprocessing(self):
        # 사용자 정의 전처리 코드
        return self.data

    @print_log("Executed {func.__name__} : shape({self.data.shape}) \n\n {self.data}")
    def transform(self):
        # 전처리 수행 코드 (위의 메소드 활용 가능)
        self.handle_missing_data()
        # self.handle_outliers()
        # self.normalize()
        # self.custom_preprocessing()
        return self.data

In [9]:
global_features = {}

In [10]:
def calculate_rsi(data, window_size=7):
    price_diff = data['wap'].diff()
    gain = price_diff.where(price_diff > 0, 0)
    loss = -price_diff.where(price_diff < 0, 0)

    avg_gain = gain.rolling(window=window_size).mean()
    avg_loss = loss.rolling(window=window_size).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi

In [11]:
@njit(parallel=True)
def compute_triplet_imbalance(df_values, comb_indices):
    """
    Calculate the triplet imbalance for each row in the DataFrame.
    :param df_values: 
    :param comb_indices: 
    :return: 
    """
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))

    for i in prange(num_combinations):
        a, b, c = comb_indices[i]
        for j in range(num_rows):
            max_val = max(df_values[j, a], df_values[j, b], df_values[j, c])
            min_val = min(df_values[j, a], df_values[j, b], df_values[j, c])
            mid_val = df_values[j, a] + df_values[j, b] + df_values[j, c] - min_val - max_val
            if mid_val == min_val:  # Prevent division by zero
                imbalance_features[j, i] = np.nan
            else:
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val + EPS)

    return imbalance_features


def calculate_triplet_imbalance_numba(price, df):
    """
    Calculate the triplet imbalance for each row in the DataFrame.
    :param price: 
    :param df: 
    :return: 
    """
    # Convert DataFrame to numpy array for Numba compatibility
    df_values = df[price].values
    comb_indices = [(price.index(a), price.index(b), price.index(c)) for a, b, c in combinations(price, 3)]

    # Calculate the triplet imbalance
    features_array = compute_triplet_imbalance(df_values, comb_indices)

    # Create a DataFrame from the results
    columns = [f"{a}_{b}_{c}_imb2" for a, b, c in combinations(price, 3)]
    features = pd.DataFrame(features_array, columns=columns)

    return features

In [12]:
weights = [
    0.004, 0.001, 0.002, 0.006, 0.004, 0.004, 0.002, 0.006, 0.006, 0.002, 0.002, 0.008,
    0.006, 0.002, 0.008, 0.006, 0.002, 0.006, 0.004, 0.002, 0.004, 0.001, 0.006, 0.004,
    0.002, 0.002, 0.004, 0.002, 0.004, 0.004, 0.001, 0.001, 0.002, 0.002, 0.006, 0.004,
    0.004, 0.004, 0.006, 0.002, 0.002, 0.04 , 0.002, 0.002, 0.004, 0.04 , 0.002, 0.001,
    0.006, 0.004, 0.004, 0.006, 0.001, 0.004, 0.004, 0.002, 0.006, 0.004, 0.006, 0.004,
    0.006, 0.004, 0.002, 0.001, 0.002, 0.004, 0.002, 0.008, 0.004, 0.004, 0.002, 0.004,
    0.006, 0.002, 0.004, 0.004, 0.002, 0.004, 0.004, 0.004, 0.001, 0.002, 0.002, 0.008,
    0.02 , 0.004, 0.006, 0.002, 0.02 , 0.002, 0.002, 0.006, 0.004, 0.002, 0.001, 0.02,
    0.006, 0.001, 0.002, 0.004, 0.001, 0.002, 0.006, 0.006, 0.004, 0.006, 0.001, 0.002,
    0.004, 0.006, 0.006, 0.001, 0.04 , 0.006, 0.002, 0.004, 0.002, 0.002, 0.006, 0.002,
    0.002, 0.004, 0.006, 0.006, 0.002, 0.002, 0.008, 0.006, 0.004, 0.002, 0.006, 0.002,
    0.004, 0.006, 0.002, 0.004, 0.001, 0.004, 0.002, 0.004, 0.008, 0.006, 0.008, 0.002,
    0.004, 0.002, 0.001, 0.004, 0.004, 0.004, 0.006, 0.008, 0.004, 0.001, 0.001, 0.002,
    0.006, 0.004, 0.001, 0.002, 0.006, 0.004, 0.006, 0.008, 0.002, 0.002, 0.004, 0.002,
    0.04 , 0.002, 0.002, 0.004, 0.002, 0.002, 0.006, 0.02 , 0.004, 0.002, 0.006, 0.02,
    0.001, 0.002, 0.006, 0.004, 0.006, 0.004, 0.004, 0.004, 0.004, 0.002, 0.004, 0.04,
    0.002, 0.008, 0.002, 0.004, 0.001, 0.004, 0.006, 0.004,
]
_weights = {int(k):v for k,v in enumerate(weights)}

In [13]:
class FeatureEngineer:

    def __init__(self, data, infer=False, feature_versions=None, dependencies=None,
                 base_directory="./data/fe_versions"):
        self.data = data
        self.infer = infer
        self.feature_versions = feature_versions or []
        self.dependencies = dependencies or {}  # 피처 버전 간 의존성을 정의하는 딕셔너리
        self.base_directory = base_directory
        if not os.path.exists(self.base_directory):
            os.makedirs(self.base_directory)

    @staticmethod
    @print_log("Executed {func_name}, Elapsed time: {elapsed_time:.2f} seconds")
    def generate_global_features(data):
        global_features["version_0"] = {
            "median_size": data.groupby("stock_id")["bid_size"].median() + data.groupby("stock_id")[
                "ask_size"].median(),
            "std_size": data.groupby("stock_id")["bid_size"].std() + data.groupby("stock_id")["ask_size"].std(),
            "ptp_size": data.groupby("stock_id")["bid_size"].max() - data.groupby("stock_id")["bid_size"].min(),
            "median_price": data.groupby("stock_id")["bid_price"].median() + data.groupby("stock_id")[
                "ask_price"].median(),
            "std_price": data.groupby("stock_id")["bid_price"].std() + data.groupby("stock_id")["ask_price"].std(),
            "ptp_price": data.groupby("stock_id")["bid_price"].max() - data.groupby("stock_id")["ask_price"].min(),
        }

    @print_log("Executed {func_name}, Elapsed time: {elapsed_time:.2f} seconds")
    def feature_selection(self, data, exclude_columns):
        # 제외할 컬럼을 뺀 나머지로 구성된 새로운 DataFrame을 생성합니다.
        selected_columns = [c for c in data.columns if c not in exclude_columns]
        data = data[selected_columns]
        return data

    @print_log("Executed {func_name}, Elapsed time: {elapsed_time:.2f} seconds")
    def feature_version_yongmin_0(self, *args, version_name="feature_version_yongmin_0"):
        
        df = pd.DataFrame(index=self.data.index)

        df['dow'] = self.data["date_id"] % 5
        df['seconds'] = self.data['seconds_in_bucket'] % 60
        df['minute'] = self.data['seconds_in_bucket'] // 60
    
        df["volume"] = self.data.eval("ask_size + bid_size")
        df['cum_wap'] = self.data.groupby(['stock_id'])['wap'].cumprod()
    
        for i in [1, 6]:
            df[f'pct_change_{i}'] = self.data.groupby(['stock_id', 'seconds_in_bucket'])['wap'].pct_change(i).fillna(0)

            f = lambda x: 1 if x > 0 else (0 if x == 0 else -1)
            df[f'polarize_pct_{i}'] = df[f'pct_change_{i}'].apply(f)
    
        return df

    @print_log("Executed {func_name}, Elapsed time: {elapsed_time:.2f} seconds")
    def feature_version_yongmin_1(self, *args, version_name="feature_version_yongmin_1"):
        
        df = pd.DataFrame(index=self.data.index)
        
        window_size = 6
        short_window = 1
        long_window = 6

        df['vol_std'] = self.data.groupby(['stock_id'])['wap'].pct_change().rolling(window=window_size).std()
        df['rolling_vol_di'] = self.data.groupby(['date_id'])['wap'].pct_change().rolling(window=window_size).std()
        df['std_st'] = self.data.groupby(['stock_id'])['wap'].rolling(window=window_size).std().values
        df['wap_pctch'] = self.data.groupby(['stock_id','date_id'])['wap'].pct_change().values*100
        df['short_ema'] = self.data.groupby(['stock_id'])['wap'].ewm(span=short_window, adjust=False).mean().values
        df['long_ema'] = self.data.groupby(['stock_id'])['wap'].ewm(span=long_window, adjust=False).mean().values
        wap_mean = self.data['wap'].mean()
        df['wap_vs_market'] = self.data['wap'] - self.data.groupby(['stock_id'])['wap'].transform('mean')
        df['macd'] = df['short_ema'] - df['long_ema']
        
        # Bollinger Bands calculation within each stock, date, and time
        df['bollinger_upper'] = self.data.groupby(['stock_id'])['wap'].rolling(window=long_window).mean().values + 2 * self.data.groupby(['stock_id'])['wap'].rolling(window=window_size).std().values
        df['bollinger_lower'] = self.data.groupby(['stock_id'])['wap'].rolling(window=long_window).mean().values - 2 * self.data.groupby(['stock_id'])['wap'].rolling(window=window_size).std().values
        # RSI calculation within each stock, date, and time
        df['rsi'] = self.data.groupby(['stock_id']).apply(calculate_rsi).values
        
        return df

    @print_log("Executed {func_name}, Elapsed time: {elapsed_time:.2f} seconds")
    def feature_version_yongmin_2(self, *args, version_name="feature_version_yongmin_2"):
        # feature engineering version 1
        # create empty dataframe
        df = pd.DataFrame(index=self.data.index)
        self.data["stock_weights"] = self.data["stock_id"].map(_weights)
        self.data["weighted_wap"] = self.data["stock_weights"] * self.data["wap"]
        df['wap_momentum'] = self.data.groupby('stock_id')['weighted_wap'].pct_change(periods=6)

        df["imbalance_momentum"] = self.data.groupby(['stock_id'])['imbalance_size'].diff(periods=1) / self.data['matched_size']
        self.data["price_spread"] = self.data["ask_price"] - self.data["bid_price"]
        
        self.data["mid_price"] = self.data.eval("(ask_price + bid_price) / 2")
        self.data["liquidity_imbalance"] = self.data.eval(f"(bid_size-ask_size)/(bid_size+ask_size+{EPS})")
        df["matched_imbalance"] = self.data.eval(f"(imbalance_size-matched_size)/(matched_size+imbalance_size+{EPS})")
        df["size_imbalance"] = self.data.eval(f"bid_size / ask_size+{EPS}")
        
        df["spread_intensity"] = self.data.groupby(['stock_id'])['price_spread'].diff()
        df['price_pressure'] = self.data['imbalance_size'] * (self.data['ask_price'] - self.data['bid_price'])
        df['market_urgency'] = self.data['price_spread'] * self.data['liquidity_imbalance']
        df['depth_pressure'] = (self.data['ask_size'] - self.data['bid_size']) * (self.data['far_price'] - self.data['near_price'])
        
        df['spread_depth_ratio'] = (self.data['ask_price'] - self.data['bid_price']) / (self.data['bid_size'] + self.data['ask_size'])
        df['mid_price_movement'] = self.data['mid_price'].diff(periods=5).apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
        
        df['micro_price'] = ((self.data['bid_price'] * self.data['ask_size']) + (self.data['ask_price'] * self.data['bid_size'])) / (self.data['bid_size'] + self.data['ask_size'])
        df['relative_spread'] = (self.data['ask_price'] - self.data['bid_price']) / self.data['wap']

        return df

    
    

    @print_log("Executed {func_name}, Elapsed time: {elapsed_time:.2f} seconds")
    def feature_version_alvin_1(self, *args, version_name="feature_version_alvin_1"):
        # feature engineering version 1
        # create empty dataframe
        df = pd.DataFrame(index=self.data.index)
        prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
        sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]

        for c in combinations(prices, 2):
            df[f"{c[0]}_{c[1]}_imb"] = self.data.eval(f"({c[0]} - {c[1]})/({c[0]} + {c[1]} + {EPS})")

        for c in [['ask_price', 'bid_price', 'wap', 'reference_price'], sizes]:
            triplet_feature = calculate_triplet_imbalance_numba(c, self.data)
            df[triplet_feature.columns] = triplet_feature.values

        return df

    @print_log("Executed {func_name}, Elapsed time: {elapsed_time:.2f} seconds")
    def feature_market_cap(self, *args, version_name="feature_market_cap"):
        df = pd.DataFrame(index=self.data.index)

        df = get_stock_info(df, self.data, "Market Cap")

        return df

    @print_log("Executed {func_name}, Elapsed time: {elapsed_time:.2f} seconds")
    def feature_sector(self, *args, version_name="feature_sector"):
        df = pd.DataFrame(index=self.data.index)

        df = get_stock_info(df, self.data, "Sector")

        return df

    @print_log("Executed {func_name}, Elapsed time: {elapsed_time:.2f} seconds")
    def feature_industry(self, *args, version_name="feature_industry"):
        df = pd.DataFrame(index=self.data.index)

        df = get_stock_info(df, self.data, "Industry")

        return df

    # you can add more feature engineering version like above
    @print_log("Executed {func_name}, Elapsed time: {elapsed_time:.2f} seconds")
    def execute_feature_versions(self, save=False, load=False):
        results = {}

        for version in self.feature_versions:
            if load:
                df = self._load_from_parquet(version)
            else:
                method = getattr(self, version, None)
                if callable(method):
                    args = []
                    for dep in self.dependencies.get(version, []):
                        dep_result = results.get(dep)
                        if isinstance(dep_result, pd.DataFrame):
                            args.append(dep_result)
                        elif dep_result is None and hasattr(self, dep):
                            dep_method = getattr(self, dep)
                            dep_result = dep_method()
                            results[dep] = dep_result
                            args.append(dep_result)
                        else:
                            args.append(None)
                    df = method(*args)
                    if save:
                        self._save_to_parquet(df, version)
            results[version] = df

        # return that was in self.feature_versions
        return {k: v for k, v in results.items() if k in self.feature_versions}

    @print_log("Executed {func_name}, Elapsed time: {elapsed_time:.2f} seconds")
    def transform(self, save=False, load=False):
        feature_versions_results = self.execute_feature_versions(save=save, load=load)
        if not self.infer:
            self.data["date_id_copy"] = self.data["date_id"]
        concat_df = pd.concat([self.data] + list(feature_versions_results.values()), axis=1)

        exclude_columns = ["row_id", "time_id", "date_id"]
        final_data = self.feature_selection(concat_df, exclude_columns)
        final_data = concat_df
        return final_data


In [14]:
class Splitter:
    """
    데이터 분리 클래스
    
    Attributes
    ----------
    method : str
        데이터 분리 방식
    n_splits : int
        데이터 분리 개수
    correct : bool
        데이터 분리 시 boundary를 맞출지 여부
    initial_fold_size_ratio : float
        초기 fold size 비율
    train_test_ratio : float
        train, test 비율
        
    Methods
    -------
    split()
        데이터 분리 수행
    """

    def __init__(self, method, n_splits, correct, initial_fold_size_ratio=0.6, train_test_ratio=0.8, gap=0,
                 overlap=True, train_start=0,
                 train_end=390, valid_start=391, valid_end=480):
        self.method = method
        self.n_splits = n_splits
        self.correct = correct
        self.initial_fold_size_ratio = initial_fold_size_ratio
        self.train_test_ratio = train_test_ratio

        self.gap = gap
        self.overlap = overlap

        # only for holdout method
        self.train_start = train_start
        self.train_end = train_end
        self.valid_start = valid_start
        self.valid_end = valid_end

        self.target = config["target"]

        self.boundaries = []

    def split(self, data):
        self.data = data #reduce_mem_usage(data)
        self.all_dates = self.data['date_id_copy'].unique()
        if self.method == "time_series":
            if self.n_splits <= 1:
                raise ValueError("Time series split method only works with n_splits > 1")
            return self._time_series_split(data)
        elif self.method == "rolling":
            if self.n_splits <= 1:
                raise ValueError("Rolling split method only works with n_splits > 1")
            return self._rolling_split(data)
        elif self.method == "blocking":
            if self.n_splits <= 1:
                raise ValueError("Blocking split method only works with n_splits > 1")
            self.initial_fold_size_ratio = 1.0 / self.n_splits
            return self._rolling_split(data)
        elif self.method == "holdout":
            if self.n_splits != 1:
                raise ValueError("Holdout method only works with n_splits=1")
            return self._holdout_split(data)
        else:
            raise ValueError("Invalid method")

    def _correct_boundary(self, data, idx, direction="forward"):
        # Correct the boundary based on date_id_copy
        original_idx = idx
        if idx == 0 or idx == len(data) - 1:
            return idx
        if direction == "forward":
            while idx < len(data) and data.iloc[idx]['date_id_copy'] == data.iloc[original_idx]['date_id_copy']:
                idx += 1
        elif direction == "backward":
            while idx > 0 and data.iloc[idx]['date_id_copy'] == data.iloc[original_idx]['date_id_copy']:
                idx -= 1
            idx += 1  # adjust to include the boundary
        return idx

    def _time_series_split(self, data):
        n = len(data)
        initial_fold_size = int(n * self.initial_fold_size_ratio)
        initial_test_size = int(initial_fold_size * (1 - self.train_test_ratio))
        increment = (1.0 - self.initial_fold_size_ratio) / (self.n_splits - 1)

        for i in range(self.n_splits):
            fold_size = int(n * (self.initial_fold_size_ratio + i * increment))
            train_size = fold_size - initial_test_size

            if self.correct:
                train_size = self._correct_boundary(data, train_size, "forward")
                end_of_test = self._correct_boundary(data, train_size + initial_test_size, "forward")
            else:
                end_of_test = train_size + initial_test_size

            train_slice = data.iloc[:train_size]
            test_slice = data.iloc[train_size:end_of_test]
            if test_slice.shape[0] == 0:
                raise ValueError("Try setting correct=False or Try reducing the train_test_ratio")

            X_train = train_slice.drop(columns=[self.target, 'date_id_copy'])
            y_train = train_slice[self.target]
            X_test = test_slice.drop(columns=[self.target, 'date_id_copy'])
            y_test = test_slice[self.target]

            self.boundaries.append((
                train_slice['date_id_copy'].iloc[0],
                train_slice['date_id_copy'].iloc[-1],
                test_slice['date_id_copy'].iloc[-1]
            ))
            yield X_train, y_train, X_test, y_test

    def _rolling_split(self, data):
        n = len(data)
        total_fold_size = int(n * self.initial_fold_size_ratio)
        test_size = int(total_fold_size * (1 - self.train_test_ratio))
        gap_size = int(total_fold_size * self.gap)
        train_size = total_fold_size - test_size
        rolling_increment = (n - total_fold_size) // (self.n_splits - 1)

        end_of_test = n - 1
        start_of_test = end_of_test - test_size
        end_of_train = start_of_test - gap_size
        start_of_train = end_of_train - train_size

        for _ in range(self.n_splits):
            if self.correct:
                start_of_train = self._correct_boundary(data, start_of_train, direction="forward")
                end_of_train = self._correct_boundary(data, end_of_train, direction="backward")
                start_of_test = self._correct_boundary(data, start_of_test, direction="forward")
                end_of_test = self._correct_boundary(data, end_of_test, direction="forward")

            train_slice = data[start_of_train:end_of_train]
            test_slice = data[start_of_test:end_of_test]
            if test_slice.shape[0] == 0:
                raise ValueError("Try setting correct=False or Try reducing the train_test_ratio")

            X_train = train_slice.drop(columns=[self.target, 'date_id_copy'])
            y_train = train_slice[self.target]
            X_test = test_slice.drop(columns=[self.target, 'date_id_copy'])
            y_test = test_slice[self.target]

            self.boundaries.append((
                train_slice['date_id_copy'].iloc[0],
                train_slice['date_id_copy'].iloc[-1],
                test_slice['date_id_copy'].iloc[0],
                test_slice['date_id_copy'].iloc[-1]
            ))
            yield X_train, y_train, X_test, y_test
            start_of_train = max(start_of_train - rolling_increment, 0)
            end_of_train -= rolling_increment
            start_of_test -= rolling_increment
            end_of_test -= rolling_increment

    def _holdout_split(self, data):
        # train_start ~ train_end : 학습 데이터 기간
        # valid_start ~ valid_end : 검증 데이터 기간
        # 학습 및 검증 데이터 분리
        threshold = int(data['date_id_copy'].nunique() * self.train_test_ratio)
        self.train_start, self.train_end = 0, data['date_id_copy'].unique()[threshold]
        self.valid_start, self.valid_end = self.train_end + self.gap, data['date_id_copy'].unique()[-1]
        
        train_mask = (data['date_id_copy'] >= self.train_start) & (data['date_id_copy'] <= self.train_end)
        valid_mask = (data['date_id_copy'] >= self.valid_start) & (data['date_id_copy'] <= self.valid_end)

        train_slice = data[train_mask]
        valid_slice = data[valid_mask]

        X_train = train_slice.drop(columns=[self.target, 'date_id_copy'])
        y_train = train_slice[self.target]
        X_valid = valid_slice.drop(columns=[self.target, 'date_id_copy'])
        y_valid = valid_slice[self.target]

        self.boundaries.append((
            train_slice['date_id_copy'].iloc[0],
            train_slice['date_id_copy'].iloc[-1],
            valid_slice['date_id_copy'].iloc[0],
            valid_slice['date_id_copy'].iloc[-1]
        ))
        yield X_train, y_train, X_valid, y_valid

    def visualize_splits(self):
        print("Visualizing Train/Test Split Boundaries")

        plt.figure(figsize=(15, 6))

        for idx, (train_start, train_end, test_start, test_end) in enumerate(self.boundaries):
            train_width = train_end - train_start + 1
            plt.barh(y=idx, width=train_width, left=train_start, color='blue', edgecolor='black')
            plt.text(train_start + train_width / 2, idx - 0.15, f'{train_start}-{train_end}', ha='center', va='center',
                     color='black', fontsize=8)

            test_width = test_end - test_start + 1
            plt.barh(y=idx, width=test_width, left=test_start, color='red', edgecolor='black')
            if test_width > 0:
                plt.text(test_start + test_width / 2, idx + 0.15, f'{test_start}-{test_end}', ha='center', va='center',
                         color='black', fontsize=8)

        plt.yticks(range(len(self.boundaries)), [f"split {i + 1}" for i in range(len(self.boundaries))])
        plt.xticks(self.all_dates[::int(len(self.all_dates) / 10)])
        plt.xlabel("date_id_copy")
        plt.title("Train/Test Split Boundaries")
        plt.grid(axis='x')

        plt.tight_layout()
        plt.show()

In [15]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.activations import gelu, softmax
from tensorflow.keras.models import Sequential

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        # parametreleri
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        # batch-layer
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TabTransformer(keras.Model):

    def __init__(self, 
            categories,
            num_continuous,
            dim,
            dim_out,
            depth,
            heads,
            attn_dropout,
            ff_dropout,
            mlp_hidden,
            normalize_continuous = True):
        
        super(TabTransformer, self).__init__()

        # --> continuous inputs
        self.normalize_continuous = normalize_continuous
        if normalize_continuous:
            self.continuous_normalization = layers.LayerNormalization()

        # --> categorical inputs

        # embedding
        self.embedding_layers = []
        for number_of_classes in categories:
            self.embedding_layers.append(layers.Embedding(input_dim = number_of_classes, output_dim = dim))

        # concatenation
        self.embedded_concatenation = layers.Concatenate(axis=1)

        # adding transformers
        self.transformers = []
        for _ in range(depth):
            self.transformers.append(TransformerBlock(dim, heads, dim))
        self.flatten_transformer_output = layers.Flatten()

        # --> MLP
        self.pre_mlp_concatenation = layers.Concatenate()

        # mlp layers
        self.mlp_layers = []
        for size, activation in mlp_hidden:
            self.mlp_layers.append(layers.Dense(size, activation=activation))

        self.output_layer = layers.Dense(dim_out)

    def call(self, inputs):
        categorical_inputs = inputs
#         print(inputs[:,0])
        # --> categorical
        embedding_outputs = []
        for i in range(len(self.embedding_layers)):
            embedding_outputs.append(tf.expand_dims(self.embedding_layers[i](categorical_inputs[:,i]),axis=1))
#         print(embedding_outputs[0].shape)
        categorical_inputs = self.embedded_concatenation(embedding_outputs)
#         categorical_inputs = tf.expand_dims(categorical_inputs,axis=1)
        for transformer in self.transformers:
            categorical_inputs = transformer(categorical_inputs)
        contextual_embedding = self.flatten_transformer_output(categorical_inputs)

        for mlp_layer in self.mlp_layers:
            mlp_input = mlp_layer(contextual_embedding)

        return self.output_layer(mlp_input)

In [16]:
dependencies = {}

In [18]:
if config["train_mode"]:
    # Read Data
    df = (pd.read_csv(f"{config['data_dir']}/train.csv")
          .drop(['time_id', 'row_id'], axis=1))

    # Feature Engineering

    feature_engineer = FeatureEngineer(df, feature_versions=['feature_version_yongmin_0', 'feature_version_yongmin_1', 'feature_version_yongmin_2',
                                                             'feature_version_alvin_1'],
                                       dependencies=dependencies)
    
    feature_engineer.generate_global_features(df)
    
    df = feature_engineer.transform()
    
    # fillna
    df = (df.replace([np.inf, -np.inf], np.nan)
          .fillna(method='ffill')
          .fillna(0)
         )

    df = reduce_mem_usage(df)

    # Set Variables Type
    categorical = ['stock_id']

    continuous = list(set(df.columns) - set(categorical) - set(['target', 'date_id_copy']))

    rubbish = gc.collect()

    BATCH_SIZE = 64
    LR = 0.001
    EPOCH = 30
    INPUT_SHAPE = df.shape[-1] - 2 # -2 for target and date_id_copy
    # nu = [df[col].nunique() for col in df.drop(['target', 'date_id_copy'], axis=1).columns]
    nu = [df[col].nunique() for col in categorical]

    ckp_path = os.path.join(config['model_dir'], 'tabtransformer_30epochV0.h5')
    if not os.path.exists(config['model_dir']):
        os.mkdir(config['model_dir'])
    
    model = TabTransformer(
                            categories = nu, # number of unique elements in each categorical feature
                            num_continuous = len(continuous),      # number of numerical features
                            dim = 16,                # embedding/transformer dimension
                            dim_out = 1,            # dimension of the model output
                            depth = 6,               # number of transformer layers in the stack
                            heads = 8,               # number of attention heads
                            attn_dropout = 0.2,      # attention layer dropout in transformers
                            ff_dropout = 0.2,        # feed-forward layer dropout in transformers
                            mlp_hidden = [(128, 'relu'), (64, 'relu')] # mlp layer dimensions and activations
                        )
    
    model.compile(AdamW(LR), 'mae', metrics=['mae'])

    splitter = Splitter(method='holdout', n_splits=1, correct=True, train_test_ratio=0.98, gap=5)
    for idx, (X_train, y_train, X_test, y_test) in enumerate(splitter.split(df)):
        print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
        
        trainds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
        trainds_batch = trainds.batch(BATCH_SIZE, drop_remainder = True)
        
        valds = tf.data.Dataset.from_tensor_slices((X_test, y_test))
        valds_batch = valds.batch(BATCH_SIZE, drop_remainder = True)

        rlr = ReduceLROnPlateau(monitor='val_mae', factor=0.5, patience=3, verbose=0, min_delta=1e-5, mode='min')
        ckp = ModelCheckpoint(ckp_path, monitor='val_mae', verbose=0, save_best_only=True, save_weights_only=True, mode='min')
        es = EarlyStopping(monitor='val_mae', min_delta=1e-4, patience=5, mode='min', restore_best_weights=True, verbose=0)
    
        model_callbacks = [rlr, ckp, es]
    
        print(f"Fitting Model - No CV")
        model.fit(trainds_batch,
                  validation_data = valds_batch,
                  batch_size = BATCH_SIZE,
                  epochs = EPOCH,
                  callbacks = model_callbacks)
        
        model.save_weights(ckp_path)
        print("Save Error")
    
        pred = model.predict(X_train, batch_size=batch_size).ravel()
        print("Train NN Score:", mean_absolute_error(y_train, pred))
    
        K.clear_session()
        del model
        rubbish = gc.collect()


----------------------------------------------------------------------------------------------------
Executed generate_global_features, Elapsed time: 0.61 seconds
----------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------
Executed feature_version_yongmin_0, Elapsed time: 2.35 seconds, shape((5237980, 9))
----------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------
Executed feature_version_yongmin_1, Elapsed time: 10.35 seconds, shape((5237980, 11))
----------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------
Executed feature_version_yongmin_2, Elapsed time: 1.37

2023-12-10 07:04:14.759023: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-10 07:04:14.769828: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-10 07:04:14.769911: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

(5138980, 74) (5138980,) (55000, 74) (55000,)
Fitting Model - No CV
Epoch 1/30


2023-12-10 07:04:21.249725: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fa3dbbb8b80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-10 07:04:21.249738: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2023-12-10 07:04:21.252945: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-10 07:04:21.263366: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-12-10 07:04:21.305975: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


80296/80296 [==============================] - 978s 12ms/step - loss: 6.4223 - mae: 6.4223 - val_loss: 5.2831 - val_mae: 5.2831 - lr: 0.0010
Epoch 2/30
80296/80296 [==============================] - 951s 12ms/step - loss: 6.4225 - mae: 6.4225 - val_loss: 5.2829 - val_mae: 5.2829 - lr: 0.0010
Epoch 3/30
80296/80296 [==============================] - 965s 12ms/step - loss: 6.4223 - mae: 6.4223 - val_loss: 5.2831 - val_mae: 5.2831 - lr: 0.0010
Epoch 4/30
80296/80296 [==============================] - 995s 12ms/step - loss: 6.4222 - mae: 6.4222 - val_loss: 5.2845 - val_mae: 5.2845 - lr: 0.0010
Epoch 5/30
80296/80296 [==============================] - 969s 12ms/step - loss: 6.4223 - mae: 6.4223 - val_loss: 5.2844 - val_mae: 5.2844 - lr: 0.0010
Epoch 6/30
80296/80296 [==============================] - 969s 12ms/step - loss: 6.4221 - mae: 6.4221 - val_loss: 5.2839 - val_mae: 5.2839 - lr: 5.0000e-04
Epoch 7/30
80296/80296 [==============================] - 985s 12ms/step - loss: 6.4221 - mae: 

NameError: name 'batch_size' is not defined

In [22]:
if MODE == "train":
    ! /usr/local/bin/kaggle datasets init -p {config['model_dir']}
    import json

    with open(f"{config['model_dir']}/dataset-metadata.json", "r") as file:
        data = json.load(file)

    data["title"] = data["title"].replace("INSERT_TITLE_HERE", f"{KAGGLE_DATASET_NAME}")
    data["id"] = data["id"].replace("INSERT_SLUG_HERE", f"{KAGGLE_DATASET_NAME}")

    with open(f"{config['model_dir']}/dataset-metadata.json", "w") as file:
        json.dump(data, file, indent=2)

    ! /usr/local/bin/kaggle datasets create -p {config['model_dir']}

    # !/usr/local/bin/kaggle datasets version -p {config['model_dir']} -m 'Updated data'

Data package template written to: ./models/20231210_16:02:31/dataset-metadata.json
Starting upload for file tabtransformer_30epochV0.h5
100%|█████████████████████████████████████████| 376k/376k [00:02<00:00, 151kB/s]
Upload successful: tabtransformer_30epochV0.h5 (376KB)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/datasets/jhk3211/tabtransformer-version-yongmin-0


In [ ]:
# df = (pd.read_csv(f"{config['data_dir']}/train.csv")
#       .drop(['time_id', 'row_id'], axis=1))

# # Feature Engineering

# feature_engineer = FeatureEngineer(df, feature_versions=['feature_version_yongmin_0', 'feature_version_yongmin_1'],
#                                    dependencies=dependencies)

# feature_engineer.generate_global_features(df)

# df = feature_engineer.transform()

# # fillna
# df = (df.replace([np.inf, -np.inf], np.nan)
#       .fillna(method='ffill')
#       .fillna(0)
#      )

# df = reduce_mem_usage(df)
# rubbish = gc.collect()

# BATCH_SIZE = 128
# LR = 0.001
# EPOCH = 30
# INPUT_SHAPE = df.shape[-1] - 2 # -2 for target and date_id_copy

# final_model = Sequential([
#                     Input(shape = (INPUT_SHAPE), batch_size=BATCH_SIZE),
#                     SAINT(3, 6, 8, 4, 4),
#                     Dense(1 , activation = 'linear')
# ])

# final_model.compile(tf.keras.optimizers.Adam(LR), 'mae', metrics=['mae'])

# model_path = "./models/20231130_11:05:26 /saint_30epoch.h5"
# final_model.load_weights(model_path)